In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
dataset = pd.read_csv("data/othello_dataset.csv")
dataset

,eOthello_game_id,winner,game_moves
0,1056798,1,f5d6c4d3e6f4e3f6c5b4e7f3c6d7b5a5c3b3g5h5g4h4e2...
1,1050515,-1,d3c5f6f5e6e3d6f7b6d7e2c6d8c4e7c3f4c8c7d2d1a6b3...
2,996989,1,f5d6c4d3c3f4e6b3e2c5c7f6g5e3e7g6g4h4f3g3f2h6h5...
3,977775,0,f5d6c5b4c3e3e2f4f3g4e6g5f6f7e7g3h6c4d3h5g6h7b3...
4,900860,1,f5d6c4d3c3f4c5b4g3c6b5f6e2g4e3e6b3a5d7e7f3a3h4...
...,...,...,...
25652,16872,-1,c4e3f5c5c3e6f4g4d6d3c6b4d2c2f3f6b3b5h4g3h3f2e2...
25653,16761,-1,c4e3f6e6f5c5c3c6d3d2f3b3b4c2b5d6e2f4f2e1c1g5g4...
25654,16740,1,f5d6c4d3c3f4f6f3e6e7d7c5b6b5c6b4e3e8g5c2f8d2d1...
25655,16906,1,f5d6c3f4f6d3c4f3e6c6c5e7e3e2f7g5g6f8h5h6h7b6b5...


In [ ]:
def process_gamemoves(move_sequence):
    moves = [move_sequence[i:i+2] for i in range(0, len(move_sequence), 2)]
    for i in range(len(moves)):
        input_move = "".join(moves[:i])
        output_move = moves[i]

        inputs.append(input_move)
        outputs.append(output_move)

inputs = []
outputs = []
dataset['game_moves'].apply(process_gamemoves)
data = {
    'input': inputs,
    'output': outputs
}
dataset = pd.DataFrame(data)
X = dataset['input']
y = dataset['output']

dataset.head()

,input,output
0,,f5
1,f5,d6
2,f5d6,c4
3,f5d6c4,d3
4,f5d6c4d3,e6


In [ ]:
X_train = X[:1200000]
X_test = X[1200000:]
y_train = y[:1200000]
y_test = y[1200000:]

model tự build sử dụng lstm

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Masking, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def index_token(move):
    if move == "":
        return -1
    col_char = move[0].lower()
    row_char = move[1]

    col = ord(col_char) - ord('a')
    row = int(row_char) - 1

    return row * 8 + col


def prompt_en_lstm(X, y):
    X_result = []
    y_result = []
    for i in range(len(y)):
        input = [X[i][j:j+2] for j in range(0, len(X[i]), 2)]
        output = y[i]

        idx_input = [index_token(move) for move in input]
        idx_output = index_token(output)

        X_result.append(idx_input)
        y_result.append(idx_output)

    X_result = pad_sequences(X_result, maxlen=60, value=-1, padding='post')
    y_result = np.array(y_result)

    return X_result, y_result

X_train_lstm, y_train_lstm = prompt_en_lstm(X_train, y_train)

In [ ]:
X_train_lstm

array([[-1, -1, -1, ..., -1, -1, -1],
       [37, -1, -1, ..., -1, -1, -1],
       [37, 43, -1, ..., -1, -1, -1],
       ...,
       [37, 43, 18, ..., -1, -1, -1],
       [37, 43, 18, ..., -1, -1, -1],
       [37, 43, 18, ..., 15, -1, -1]], dtype=int32)

In [ ]:
y_train_lstm

array([37, 43, 26, ..., 62, 15,  7])

In [ ]:
def build_model():
    model = Sequential([
        Masking(mask_value=-1, input_shape=(60,)),

        Embedding(input_dim=64, output_dim=256),

        LSTM(512, return_sequences=False),
        Dropout(0.2),
        BatchNormalization(),

        Dense(512, activation='relu'),
        Dropout(0.2),

        Dense(64, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


lstm = build_model()
lstm.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'embedding' (of type Embedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 60)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 60, 256)        │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 512)            │     1,574,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        32,832 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,888,832 (7.21 MB)

 Trainable params: 1,887,808 (7.20 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lstm.fit(
    X_train_lstm, y_train_lstm,
    epochs=20,
    batch_size=2048,
    validation_split=0.1,
    callbacks=[early_stopping],
    verbose=1
)

lstm.save('built_lstm.keras')

Epoch 1/20
257/528 ━━━━━━━━━━━━━━━━━━━━ 1:36 356ms/step - accuracy: 0.0436 - loss: 3.8324

model finetune sử dụng llm

In [ ]:
!pip install -qq -U transformers datasets

import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, TrainerCallback
from datasets import Dataset
import torch
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
def prompt_en_llm(X, y):
    prompts = []
    responses = []
    for i in range(len(y)):
        moves_seq = ' '.join([X[i][j:j+2] for j in range(0, len(X[i]), 2)])
        prompts.append(f"Played moves: {moves_seq}\nBest Next move:")
        responses.append(f" {y[i]}")
    return pd.DataFrame({'prompt': prompts, 'response': responses})

train_df = prompt_en_llm(X_train, y_train)
train_dataset = Dataset.from_pandas(train_df)

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
gpt2 = AutoModelForCausalLM.from_pretrained('gpt2')

def tokenize_function(examples):
    input_texts = examples['prompt']
    target_texts = examples['response']
    return tokenizer(input_texts, text_target=target_texts, truncation=True, padding=True, max_length=256)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

class ClearCacheCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % args.save_steps == 0:
            torch.cuda.empty_cache()
            print(f"Cleared CUDA cache")

gpt2.gradient_checkpointing_enable()



In [ ]:
training_args = TrainingArguments(
    report_to=[],
    output_dir="./finetuned_gpt2",
    save_strategy="steps",
    save_steps=500,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    save_total_limit=3,
    eval_strategy="no",
    warmup_steps=50,
    weight_decay=0.01,
    fp16=True,
    optim="adamw_torch_fused",
)

trainer = Trainer(
    model=gpt2,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    callbacks=[ClearCacheCallback()],
)

trainer.train()




In [ ]:
def deindex_token(index):
    if index < 0 or index >= 64:
        return ""
    row = index // 8
    col = index % 8
    return chr(col + ord('a')) + str(row + 1)

def infer_llm(input_moves_str):
    input_moves = [input_moves_str[i:i+2] for i in range(0, len(input_moves_str), 2)]
    prompt = f"Played moves: {' '.join(input_moves)}\nBest Next move:"
    inputs = tokenizer(prompt, return_tensors='pt').to(gpt2.device)

    outputs = gpt2.generate(
        **inputs,
        max_new_tokens=5,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(outputs[0])
    try:
        move_part = generated_text.split("Best Next move:")[1].strip()
        predicted_move = move_part.split()[0]
    except IndexError:
        predicted_move = ""
    return predicted_move

def eval_metrics(label, pred):
    print(f'accuracy: {accuracy_score(label, pred)}')
    print(f'recall: {recall_score(label, pred, average="macro")}')
    print(f'precision: {precision_score(label, pred, average="macro")}')
    print(f'f1: {f1_score(label, pred, average="macro")}')

# print('---LSTM performance---')
# y_pred = np.argmax(lstm.predict(X_test_lstm), axis=1)
# y_pred_moves = [deindex_token(idx) for idx in y_pred]
# eval_metrics(y_test, y_pred_moves)

# print('---LLM performance---')
# y_pred_llm = [infer_llm(x) for x in X_test]
# eval_metrics(y_test[:len(y_pred_llm)], y_pred_llm)


In [ ]:
!zip -r /content/gpt2.zip /content/gpt2-othello/
from google.colab import files
files.download("/content/gpt2.zip")